In [1]:
import tensorflow as tf
import numpy as np
import random
import os
import sys
import tqdm
import csv
from PIL import Image
import cv2
from matplotlib import pyplot as plt
import gc

In [2]:
def image_load_and_batch(img_list):
    im = []
    
    for img_name in img_list:
        
        img =cv2.imread(img_name)
        b, g, r = cv2.split(img)
        im.append(cv2.merge([r,g,b]))
        

            
    return im

In [3]:
def sport_label_load(label_list):
    output = []
    label_img = image_load_and_batch(label_list)
    for label in label_img:
        label = cv2.resize(label,(268,480))
        ball_posi = np.nonzero(label)
        out = np.concatenate((np.ones((268,480,1)),np.zeros((268,480,1))), axis=2)
     
        
        for i in range(len(ball_posi[0])):
            out[ball_posi[1][i], ball_posi[0][i]] = [0., 1.]
        output.append(out)
    
    #print(np.shape(output))
    return np.array(output, dtype=np.float32)
            

In [4]:
def label_make_batch(label_list):
    output = []
    
    for label in label_list:
        
        x = label[1]
        y = label[0]
        if x == None or y == None:
            
            output.append(np.concatenate((np.ones((268,480,1)),np.zeros((268,480,1))), axis=2))
            
        else:

            x = int((x/4) + 0.5)
            y = int((y/4) + 0.5)
            ball_map1 = np.ones((268,480,1))
            ball_map1[y-2:y+2,x-2:x+2] = 0
            ball_map2 = np.zeros((268,480,1))
            ball_map2[y-2:y+2,x-2:x+2] = 1
            last = np.concatenate((ball_map1,ball_map2),axis=2)
            
            output.append(last)

    
    return output
            

In [5]:
def make_batch(all_data):
    
    img_list = []
    label_list = []
    
    keys = list(all_data.keys())
    random.shuffle(keys)
    for key in keys:
        img_list.append(key)
        label_list.append(all_data[key])
    
    train_img = img_list[:10000]
    test_img = img_list[10000:]
    
    train_label = label_list[:10000]
    test_label = label_list[10000:]
    
    return train_img, test_img, train_label, test_label

In [6]:
def Conv_block(data, train=True, filter1=3, filter2=3, maxpool=True, std =1, in_ch=3, out_ch=3):    
    
    
    conv1 = tf.layers.conv2d(data, filters=out_ch,kernel_size=filter1, strides=std, padding="SAME", trainable=train)
    output = tf.layers.conv2d(conv1, filters=out_ch,kernel_size=filter2, strides=1, padding='SAME',trainable=train)
    if maxpool:
        output = tf.layers.max_pooling2d(inputs = output, pool_size=2, strides=2, padding='VALID')
    
    return output

In [10]:
def Upsample_block(data, output_size, kernel=2, std=2):
    
    output = tf.layers.conv2d_transpose(data, filters=output_size, kernel_size=kernel, strides=std, padding="VALID")
    
    return output

In [9]:
def Model(data, label, train, batch_size):

    
    
    with tf.variable_scope("conv1"):
        Conv1_feature = Conv_block(data, train, filter1=7, filter2=3, maxpool=True, std=2,in_ch=3,out_ch=8)
        Conv1_feature = tf.slice(Conv1_feature,[0,0,0,0],[batch_size,268,480,8])
        print("Conv1 Done")

    with tf.variable_scope("conv2"):
        Conv2_feature = Conv_block(Conv1_feature, train, filter1=3, filter2=3, maxpool=True, in_ch=8,out_ch=16)
        print("Conv2 Done")
    with tf.variable_scope("conv3"):
        Conv3_feature = Conv_block(Conv2_feature, train, filter1=3, filter2=3, maxpool=True, in_ch=16,out_ch=32)
        print("Conv3 Done")
        
     
    
    upsize = Conv1_feature.get_shape().as_list()[-1]      
    with tf.variable_scope("Upsize1"):
        Conv2_up = Upsample_block(Conv2_feature, upsize, kernel=2, std=2)        
        print("Upsize1 Done")
    with tf.variable_scope("Upsize2"):
        upsize2 = Conv2_feature.get_shape().as_list()[-1]
        Conv3_up = Upsample_block(Conv3_feature, upsize, kernel=4, std=4)
        print("Upsize2 Done")
        '''bias = tf.zeros([batch_size, 2, 480, 8])
        Conv3_up = tf.concat([Conv3_up, bias], axis=1)'''

        
        
    with tf.variable_scope("Concat_conv"):
        Conv_concat = tf.concat([Conv1_feature, Conv2_up, Conv3_up],axis=3)  
        print("Concat Done")
    with tf.variable_scope("conv4"):
        Conv4_feature = Conv_block(Conv_concat, train, filter1=3, filter2=3, maxpool=False, in_ch = 56,out_ch=2)
        print("Conv4 Done")
        #print(Conv4_feature)


    return Conv4_feature

In [7]:
def optimizer(data, label, lr=0.0001):
    #print(data)
    logits = tf.nn.softmax(data)
    print("lgits done")
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=data, labels=label)
    print("entropy done")
    loss_op = tf.reduce_mean(cross_entropy, name="Loss")
    print("loss Done")
    train_op = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss_op, name="Train_op")
    print("train Done")
    
    return logits, loss_op, train_op
    

In [8]:
# Data Load Part
# Data Annotate In Deep_Ball_Data/Annote/ and image Deep_Ball_Data/img
#For Valley Ball


#Annotate Data Load
#f = open("./Deep_Ball_Data/Annote/cam.csv", 'r', encoding='utf-8')
f = open("./Deep_Ball_Data/Annote/cam.csv", 'r')
k = 4
rdr = csv.reader(f)
Data = []
for line in rdr:
    if line[0][0] == 'F':continue
    if line[1] == ' -': 
        Data.append([None, None])
    else:
        x = int(line[1])
        y = int(line[2])
        Data.append([x,y])
                    
f.close()

#Image Data Load

path = "./Deep_Ball_Data/img/graz-arbesbach_5"
img_list = [os.path.join(path , x) for x in os.listdir(path)]


#merge all data Want to shuffle
all_data = {}
for i,img in enumerate(img_list[:9000]):
    
    all_data[img]= Data[i]
    
#Test_data make_file
file_path = "./Deep_Ball_Result/"
#f=open(file_path + "Test.txt", 'w', encoding='utf-8')
f=open(file_path + "Test.txt", 'w')
for i in img_list[9000:]:
    f.write(i)
    f.write("\n")
f.close()


In [11]:
path = "DataSet(190408)"
path = [os.path.join(path, x) for x in os.listdir(path)]
path2 = [os.path.join(x, y) for x in path if os.path.isdir(x) for y in os.listdir(x)]
path3= [os.path.join(x, y) for x in path2 if x[-3:] =="pre" for y in os.listdir(x)]
path2 = [os.path.join(x, y) for x in path3 if os.path.isdir(x) for y in os.listdir(x)]
path3= [os.path.join(x, y) for x in path2 if os.path.isdir(x) for y in os.listdir(x)]
path2= [os.path.join(x, y) for x in path3 if x[-3:] =="img" for y in os.listdir(x)]

all_data = {}
for path in path2[:11700]:
    all_data[path] = path.replace("img", "label")

In [12]:
len(path2)

117492

In [15]:
n_epoch = 10
batch = 2000
batch_size = 16
lr = 0.001
total_loss = 0
memory_fraction = 0.7
train=True
argu_key = ['fr_ud','fr_lr','ro_90','ro_180','Gau', 'ori']




tf.reset_default_graph()


input_image = tf.placeholder(tf.float32,[None, 1080, 1920, 3])
label = tf.placeholder(tf.float32,[None, 268, 480, 2])
argu = tf.placeholder(tf.int32, [])





output = Model(input_image, label, train, batch_size)
logits, loss, train_op = optimizer(output, label, lr)
saver = tf.train.Saver(max_to_keep = 8000000)
print("Training.. Start..")


with tf.Session(config=tf.ConfigProto(gpu_options=(tf.GPUOptions(per_process_gpu_memory_fraction=memory_fraction)), log_device_placement=True)) as sess:


    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
   
    for epoch in range(n_epoch):
        counter = 0
        batch = 0
        total_loss = 0
        total_acc = 0
        Train_image, Test_image, Train_label, Test_label = make_batch(all_data)
        
        

        while True:
            print("train epoch", epoch, "train batch :", batch,"~",batch +batch_size)

            train=True


            now_img = image_load_and_batch(Train_image[batch:batch +batch_size])
            print(Train_label[batch:batch +batch_size])
            now_label = sport_label_load(Train_label[batch:batch +batch_size])
            
            
            #print(np.shape(now_img), np.shape(now_label))
            #for vallyball
            #now_label = label_make_batch(Train_label[batch:batch +batch_size])

            

            now_total_pixcel = np.sum(now_label)
            
            

            #try:
            
            logit, losses, _ = sess.run([logits, loss, train_op], feed_dict={input_image:now_img, label:now_label})           
            counter += 1
            pred = np.argmax(logit, axis=3)
            truth = np.argmax(now_label, axis =3)
            now_acc= (1-(np.shape(np.nonzero(pred-truth))[-1]/now_total_pixcel)) * 100
            print(now_acc)
            total_loss += losses
            total_acc += now_acc
            del now_img,now_label, logit, losses
            gc.collect()




            batch = batch+batch_size


            if batch==5000 or batch==10000:

                print("Now...",str(epoch),"... ",str(batch) ,"..End..")
                print("Now Loss..", str(total_loss), " Now Accuracy..", str(total_acc / counter))
                print("Validation Test Start..")

                counter =0
                test_batch = 0
                total_loss = 0
                total_acc = 0
                train = False

                while True:

                    print("test_batch : ", test_batch,"~",test_batch + batch_size)
                    test_img = image_load_and_batch(Test_image[test_batch:test_batch+ batch_size])
                    test_label = sport_label_load(Test_label[test_batch:test_batch+ batch_size])
                    #for vallyball
                    #test_label = label_make_batch(Test_label[test_batch:test_batch+ batch_size])
                    #print(np.shape(test_img))
                    #print(np.shape(test_label))

                    try:

                        test_logit, test_losses, test_train_op = sess.run([logits, loss, train_op], feed_dict={input_image:test_img, label:test_label})

                        pred = np.argmax(test_logit, axis=3)
                        truth = np.argmax(test_label, axis =3)
                        now_acc= (1-(np.shape(np.nonzero(pred-truth))[-1]/now_total_pixcel)) * 100

                        print(now_acc)
                        counter += 1
                        test_batch += batch_size
                        total_acc += now_acc
                        total_loss += test_losses

                        del test_img, test_label
                        gc.collect()


                        if test_batch>990:

                            print("Validation Test End..")
                            print("Now Loss..", str(total_loss), "Now Accuracy..", str(total_acc / counter))
                            saver.save(sess, './Deep_Ball_Result/Deep_Ball_Model3_' + str(epoch)+'_'+str(batch) + '.ckpt')
                            break

                    except TypeError:
                        test_batch += batch_size
                        if test_batch>990:

                            print("Validation Test End..")
                            print("Now Loss..", str(total_loss), "Now Accuracy..", str(total_acc / counter))
                            saver.save(sess, './Deep_Ball_Result/Deep_Ball_Model3_' + str(epoch)+'_'+str(batch) + '.ckpt')
                            break
                        continue
                    








            if batch>9999: break




Conv1 Done
Conv2 Done
Conv3 Done
Upsize1 Done
Upsize2 Done
Concat Done
Conv4 Done
lgits done
entropy done
loss Done
train Done
Training.. Start..
train epoch 0 train batch : 0 ~ 16
['DataSet(190408)\\Free_A\\Deep_pre\\00026\\1\\label\\3972flipud.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00007\\0\\label\\1120gamma.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00009\\3\\label\\1522rot_180.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00006\\3\\label\\1072rot_180.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00020\\2\\label\\3140rot_180.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00012\\1\\label\\1842ori.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00017\\1\\label\\2521gamma.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00002\\2\\label\\362ori.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00023\\2\\label\\3641rot_90.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00019\\0\\label\\2852ori.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00017\\2\\label\\2602flipud.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00026\\0\\label\\3931

['DataSet(190408)\\Free_A\\Deep_pre\\00025\\3\\label\\3922gamma.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00014\\1\\label\\2063ori.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00025\\1\\label\\3863rot_90.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00024\\1\\label\\3721flipud.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00013\\3\\label\\2020flipud.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00005\\1\\label\\793rot_90.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00021\\3\\label\\3432rot_180.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00020\\0\\label\\3022gau.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00021\\3\\label\\3433ori.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00019\\3\\label\\2931gau.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00003\\0\\label\\501ori.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00014\\2\\label\\2091flipud.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00001\\2\\label\\203rot_270.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00014\\3\\label\\2122ori.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\000

['DataSet(190408)\\Free_A\\Deep_pre\\00006\\2\\label\\982rot_180.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00002\\3\\label\\442rot_270.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00022\\2\\label\\3513rot_270.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00020\\2\\label\\3202gamma.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00008\\1\\label\\1262ori.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00008\\1\\label\\1253gamma.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00003\\2\\label\\583rot_90.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00026\\1\\label\\3971flipud.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00008\\1\\label\\1263ori.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00023\\3\\label\\3682ori.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00013\\3\\label\\2020rot_90.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00001\\0\\label\\141gamma.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00012\\3\\label\\1901gamma.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00008\\0\\label\\1242flipud.jpg', 'DataSet(190408)\\Free_A\\Deep_pr

['DataSet(190408)\\Free_A\\Deep_pre\\00007\\2\\label\\1171rot_180.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00021\\1\\label\\3383flipud.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00024\\3\\label\\3802rot_90.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00020\\3\\label\\3220gau.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00008\\0\\label\\1221rot_90.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00008\\2\\label\\1291gau.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00008\\3\\label\\1310rot_180.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00006\\0\\label\\871rot_90.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00021\\2\\label\\3392rot_180.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00001\\3\\label\\210rot_270.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00015\\0\\label\\2173gau.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00020\\0\\label\\3023rot_180.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00002\\1\\label\\271gamma.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00009\\0\\label\\1390flipud.jpg', 'DataSet(190408)\\Free_A\\D

['DataSet(190408)\\Free_A\\Deep_pre\\00008\\3\\label\\1332flipud.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00022\\0\\label\\3452rot_90.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00025\\3\\label\\3901rot_270.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00000\\1\\label\\52rot_270.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00001\\3\\label\\232rot_90.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00022\\0\\label\\3462rot_90.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00007\\3\\label\\1200rot_180.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00014\\1\\label\\2071rot_90.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00003\\0\\label\\520gamma.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00025\\1\\label\\3853gamma.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00000\\3\\label\\113rot_270.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00022\\0\\label\\3470rot_180.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00024\\0\\label\\3701gau.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00009\\0\\label\\1410gamma.jpg', 'DataSet(190408)\\Free_A\

...

['DataSet(190408)\\Free_A\\Deep_pre\\00020\\2\\label\\3081gamma.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00020\\2\\label\\81gau.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00020\\0\\label\\21rot_270.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00020\\1\\label\\3060rot_180.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00017\\2\\label\\2580rot_180.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00024\\3\\label\\3801gau.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00020\\2\\label\\3152ori.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00014\\3\\label\\2123rot_90.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00019\\0\\label\\2853gau.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00022\\3\\label\\3563rot_270.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00003\\0\\label\\523gau.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00009\\0\\label\\1382flipud.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00019\\1\\label\\2890flipud.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00020\\2\\label\\3112rot_180.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\

['DataSet(190408)\\Free_A\\Deep_pre\\00000\\3\\label\\113ori.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00012\\0\\label\\1792rot_270.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00010\\2\\label\\1612rot_270.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00009\\3\\label\\1503rot_270.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00017\\1\\label\\2550flipud.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00018\\3\\label\\2822rot_180.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00023\\2\\label\\3631rot_270.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00019\\1\\label\\2893flipud.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00025\\2\\label\\3890rot_180.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00019\\2\\label\\2901rot_270.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00022\\1\\label\\3501gamma.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00020\\2\\label\\3193flipud.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00001\\3\\label\\223flipud.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00002\\2\\label\\322rot_90.jpg', 'DataSet(190408)\\F

['DataSet(190408)\\Free_A\\Deep_pre\\00017\\2\\label\\2571rot_180.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00007\\0\\label\\1112ori.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00010\\0\\label\\1572rot_90.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00024\\2\\label\\3770gau.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00020\\2\\label\\3172rot_270.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00020\\1\\label\\3061ori.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00025\\0\\label\\3810ori.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00011\\0\\label\\1692gamma.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00021\\2\\label\\3411rot_90.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00005\\0\\label\\741ori.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00026\\1\\label\\3983rot_180.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00000\\2\\label\\62rot_270.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00015\\2\\label\\2211gamma.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00002\\3\\label\\421rot_90.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\

['DataSet(190408)\\Free_A\\Deep_pre\\00002\\2\\label\\323rot_270.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00007\\1\\label\\1152flipud.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00002\\3\\label\\462rot_270.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00011\\0\\label\\1691rot_270.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00017\\0\\label\\2440rot_90.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00019\\3\\label\\2930rot_270.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00018\\2\\label\\2801rot_180.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00017\\2\\label\\2612flipud.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00015\\1\\label\\2182rot_90.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00005\\3\\label\\851gau.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00016\\0\\label\\2291rot_180.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00009\\0\\label\\1382rot_180.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00005\\1\\label\\790gau.jpg', 'DataSet(190408)\\Free_A\\Deep_pre\\00010\\1\\label\\1591rot_270.jpg', 'DataSet(190408)\\Fre

ValueError: not enough values to unpack (expected 3, got 0)